In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date, datetime, timedelta
import time
import yfinance as yf
from pyetfdb_scraper import etf
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
import zipfile
from io import BytesIO

from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from statsmodels.graphics.tsaplots import plot_acf


from sklearn.linear_model import LinearRegression

In [39]:
df = yf.Ticker('VXX').history(period='max',  interval='1wk')

df = df.drop(['High', 'Low','Dividends','Stock Splits','Capital Gains'],axis=1)
df['return_close'] = df['Close'].pct_change()
df['return_close_lag'] = df['return_close'].shift(1)

In [40]:
px.scatter(df, x='return_close_lag', y='return_close')

In [41]:
df[['return_close_lag', 'return_close']].corr()

,return_close_lag,return_close
return_close_lag,1.000000,0.113836
return_close,0.113836,1.000000


In [42]:
px.line(df, x=df.index, y = 'return_close')

In [11]:
df['return_close_2'] = df['Close'].pct_change(2)
df['return_close_3'] = df['Close'].pct_change(3)
df['return_close_4'] = df['Close'].pct_change(4)

In [12]:
df[['return_close', 'return_close_2', 'return_close_3', 'return_close_4']].corr()

,return_close,return_close_2,return_close_3,return_close_4
return_close,1.000000,0.714962,0.589728,0.502971
return_close_2,0.714962,1.000000,0.824555,0.714956
return_close_3,0.589728,0.824555,1.000000,0.872050
return_close_4,0.502971,0.714956,0.872050,1.000000


In [47]:
px.histogram(df[df['return_close_lag'] > 0.05],x='return_close')

In [55]:
df[df['return_close_lag'] < 0.1]['return_close'].mean() 

-0.008856627177374937